In [1]:
import sys
sys.path.append('utilities/')

import multiprocessing as mp
import numpy as np
import os

# Check the maximum number of threads available on your PC

In [2]:
print("Maximum number of threads available  ", mp.cpu_count())

Maximum number of threads available   4


# ---------- USER PROVIDED INFORMATION -------------

## imputes needed:
 - DataFilePath: path to the folder containing the xlsx sheets
 - outputFolder: folder we are interesting in outputing the results in
 - data_type: cam also be empty ('') else it denotes the type of data we are analysing
         
 - colision: 1 if we want to allow overlapping of polymerase 0 otherwise
 - combined: 1 if all the xlsx in the folder are for the same phenotype, 0 otherwise
 - numberOfWorkers: it's recomended to use less than half of the total threads available 

## parameters used in the movies:
 - retention: time in seconds that the MS2 signal stays in the TS after finishing transcribing
 - Intensity_for_1_Polym =1 if the data are already divided by the calibration factor
 - Polym_speed: Polymerase speed
 - TaillePreMarq: length of mRNA until we reach the beginning of the MS2 sequence in bp
 - TailleSeqMarq: length of MS2 sequence in bp
 - TaillePostMarq: length of mRNA from last loop of MS2 until the polymerase leaves the TS
 - EspaceInterPolyMin: minimal distance between polymerase
 - FrameLen: frame length in seconds or time resolution

In [5]:
DataFilePath = 'sample_data1/rawData/'  ## path to the folder containing the data
outputFolder= 'sample_data1/output/'
data_type = ''
extension ='.xlsx'

combined = 1 
numberOfWorkers = 2
colision = 0
retention = 0 # in seconds

##### parameters
   
Intensity_for_1_Polym = 1 # calibration factor
Polym_speed = 45 # Enter the Polymerase speed'
TaillePreMarq = 0 # length of mRNA until we reach the beginning of the MS2 sequence in bp
TailleSeqMarq = 1292 # length of MS2 sequence in bp
TaillePostMarq = 4526 + Polym_speed*retention # length of mRNA from last loop of MS2 until the polymerase leaves the TS
EspaceInterPolyMin = 30 # minimal distance between polymerase
FrameLen = 3.9 # frame length in seconds or time resolution

if not os.path.exists(outputFolder):
    os.mkdir(outputFolder)  

In [6]:
npzFilePath=outputFolder
parameterFileName='drosoParameters'+ data_type #should always end with Parameters

FreqEchImg = 1/FrameLen 
DureeSignal = (TaillePreMarq + TailleSeqMarq + TaillePostMarq) / Polym_speed
FreqEchSimu = 1/(EspaceInterPolyMin/Polym_speed)

np.savez(npzFilePath+parameterFileName+'.npz', 
          Polym_speed = Polym_speed,  
          TaillePreMarq = TaillePreMarq,
          TailleSeqMarq = TailleSeqMarq,
          TaillePostMarq = TaillePostMarq,
          EspaceInterPolyMin = EspaceInterPolyMin,
          FrameLen = FrameLen,
          Intensity_for_1_Polym = Intensity_for_1_Polym,
          FreqEchImg = FreqEchImg,
          DureeSignal = DureeSignal,
          FreqEchSimu = FreqEchSimu,
          retention = retention
        )

# Import the preprocessing package 

## please chose between these three type:
####   1- readDrosoData_2spot_1st_activation
####   2- readDrosoData_2spot_2nd_activation
####  3- readDrosoData_2spot_all_nuclei
####   4- readDrosoData_1spot

In [7]:
from readDrosoData_1spot import readDataInFile

drosoSNAdata = readDataInFile(DataFilePath,outputFolder,data_type + '/', extension)

File 1: data_D3_2S.xlsx


C:\Users\ovidiu\Dropbox\BurstDECONV_supp\code_BurstDECONV\DeconvolutionPython+Notebook\utilities\readDrosoData_1spot.py:45: FutureWarning: Defining usecols with out of bounds indices is deprecated and will raise a ParserError in a future version.
  DataExp =pd.read_excel(DataFileName,usecols=np.arange(4,300), engine='openpyxl').to_numpy()


# Starting the deconvolution of the signal

In [8]:
from deconvolveMyData import deconvolveMyData


#----------------- Run deconvolution----------------------#

DataFilePathToPreProcessedFile = os.path.join(outputFolder,'npzFile' +data_type + '/')



deconvolveMyData(DataFilePath = DataFilePathToPreProcessedFile, outputFolder=outputFolder, parameterFile=npzFilePath+parameterFileName+ '.npz',
                   number_of_workers=numberOfWorkers,data_type=data_type + '/', colision = colision)


start of the genetic algorithm for file data_dataD32S.npz

Processing cell number 2 out of 4

Processing cell number 1 out of 4


C:\Users\ovidiu\Dropbox\BurstDECONV_supp\code_BurstDECONV\DeconvolutionPython+Notebook\utilities\pygadForDeconvolution.py:331: UserWarning: The percentage of genes to mutate (mutation_percent_genes=0.03) resutled in selecting (0) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).
If you do not want to mutate any gene, please set mutation_type=None.
  if not self.suppress_warnings: warnings.warn("The percentage of genes to mutate (mutation_percent_genes={mutation_percent}) resutled in selecting ({mutation_num}) genes. The number of genes to mutate is set to 1 (mutation_num_genes=1).\nIf you do not want to mutate any gene, please set mutation_type=None.".format(mutation_percent=mutation_percent_genes, mutation_num=mutation_num_genes))


ga stopped because the TolFun was reached at generation = 112
change = 0.0

Done processing cell number  2
Total time to process cell 2 is 105.36154589999933

Processing cell number 3 out of 4
ga stopped because the TolFun was reached at generation = 178
change = 0.0

Done processing cell number  1
Total time to process cell 1 is 168.43745040000067

Processing cell number 4 out of 4
ga stopped because the TolFun was reached at generation = 188
change = 3.741909971893474e-18

Done processing cell number  3
Total time to process cell 3 is 205.66029410000192
ga stopped because the TolFun was reached at generation = 172
change = 0.0

Done processing cell number  4
Total time to process cell 4 is 151.5589112000016
Results saved in sample_data1/output/resultDec/result_dataD32S.npz


# Modeling 

In [11]:
from common_part_fitting import fit

# #----------------- Fit model-----------------------#


pathToDeconvolutionResultsFolder=os.path.join(outputFolder,'resultDec'+data_type + '/')
pardor=npzFilePath+parameterFileName+ '.npz'  #using the whole path
parameterPath=npzFilePath+parameterFileName+ '.npz'
FitResults=fit(pathToDeconvolutionResultsFolder,parameterPath,combined,outputFolder,data_type + '/')

sample_data1/output/resultDec/
iteration nbr  0
iteration nbr  1
iteration nbr  2
iteration nbr  3
iteration nbr  4
iteration nbr  5
iteration nbr  6
iteration nbr  7
iteration nbr  8
iteration nbr  9
iteration nbr  10


C:\Users\ovidiu\Dropbox\BurstDECONV_supp\code_BurstDECONV\DeconvolutionPython+Notebook\utilities\common_fit2_part.py:45: RuntimeWarning: invalid value encountered in log
  return np.abs(np.log(k[2]*np.exp(k[0]*xs)+(1-k[2])*np.exp(k[1]*xs) ) -np.log(1-fs)) /sN # k: parameters


iteration nbr  11
iteration nbr  12
iteration nbr  13
iteration nbr  14
iteration nbr  15
iteration nbr  16
iteration nbr  17
iteration nbr  18
iteration nbr  19
iteration nbr  20
iteration nbr  21
iteration nbr  22
iteration nbr  23
iteration nbr  24
iteration nbr  25
iteration nbr  26
iteration nbr  27
iteration nbr  28
iteration nbr  29
iteration nbr  30
iteration nbr  31
iteration nbr  32
iteration nbr  33
iteration nbr  34
iteration nbr  35
iteration nbr  36
iteration nbr  37
iteration nbr  38
iteration nbr  39
iteration nbr  40
iteration nbr  41
iteration nbr  42
iteration nbr  43
iteration nbr  44
iteration nbr  45
iteration nbr  46
iteration nbr  47
iteration nbr  48
iteration nbr  49
iteration nbr  50
iteration nbr  51
iteration nbr  52
iteration nbr  53
iteration nbr  54
iteration nbr  55
iteration nbr  56
iteration nbr  57
iteration nbr  58
iteration nbr  59
iteration nbr  60
iteration nbr  61
iteration nbr  62
iteration nbr  63
iteration nbr  64
iteration nbr  65
iteration 

C:\Users\ovidiu\Dropbox\BurstDECONV_supp\code_BurstDECONV\DeconvolutionPython+Notebook\utilities\common_fit3_part.py:44: RuntimeWarning: invalid value encountered in log
  return (np.log(k[3]*np.exp(k[0]*xs)+k[4]*np.exp(k[1]*xs)+(1-k[3]-k[4])*np.exp(k[2]*xs) ) -np.log(1-fs)) /sN # k: parameters


iteration nbr  3
iteration nbr  4
iteration nbr  5
iteration nbr  6
iteration nbr  7
iteration nbr  8
iteration nbr  9
iteration nbr  10
iteration nbr  11
iteration nbr  12
iteration nbr  13
iteration nbr  14
iteration nbr  15
iteration nbr  16
iteration nbr  17
iteration nbr  18
iteration nbr  19
iteration nbr  20
iteration nbr  21
iteration nbr  22
iteration nbr  23
iteration nbr  24
iteration nbr  25
iteration nbr  26
iteration nbr  27
iteration nbr  28
iteration nbr  29
iteration nbr  30
iteration nbr  31
iteration nbr  32
iteration nbr  33
iteration nbr  34
iteration nbr  35
iteration nbr  36
iteration nbr  37
iteration nbr  38
iteration nbr  39
iteration nbr  40
iteration nbr  41
iteration nbr  42
iteration nbr  43
iteration nbr  44
iteration nbr  45
iteration nbr  46
iteration nbr  47
iteration nbr  48
iteration nbr  49
iteration nbr  50
iteration nbr  51
iteration nbr  52
iteration nbr  53
iteration nbr  54
iteration nbr  55
iteration nbr  56
iteration nbr  57
iteration nbr  58

parameters [ 0.0043421  -0.00833283  0.02508589  0.00118606  0.1067573 ]
ind of obj 130
parameters [ 0.00433712 -0.00833058  0.02508812  0.00118612  0.10675738]
ind of obj 131
parameters [-0.04650015  0.31165889 -0.1200698   0.00067803  0.10683971]
ind of obj 132
parameters [-0.04663673  0.31145598 -0.11972475  0.00067912  0.10684154]
ind of obj 133
parameters [-0.04667028  0.31140722 -0.11964048  0.00067935  0.1068429 ]
ind of obj 134
parameters [-0.04687181  0.31112642 -0.11914519  0.00068104  0.10684479]
ind of obj 135
parameters [-0.04187936  0.31909852 -0.13254516  0.00063328  0.10662831]
ind of obj 136
parameters [-0.041955    0.31894999 -0.13231544  0.00063395  0.10662975]
ind of obj 137
parameters [-0.04229913  0.31825173 -0.13125885  0.00063698  0.10663327]
ind of obj 138
parameters [-0.04230759  0.31823549 -0.13123357  0.00063706  0.10663338]
ind of obj 139
parameters [-0.04239091  0.31803825 -0.13095921  0.0006377   0.10663378]
ind of obj 140
parameters [-0.04246513  0.31789

C:\Users\ovidiu\Dropbox\BurstDECONV_supp\code_BurstDECONV\DeconvolutionPython+Notebook\utilities\fit3M2Parameters.py:89: RuntimeWarning: invalid value encountered in sqrt
  kk1p = 1/2 * ( -L1+S2/S1 +  np.sqrt((S1*L1-S2) **2-4*L3*S1)/S1 )
C:\Users\ovidiu\Dropbox\BurstDECONV_supp\code_BurstDECONV\DeconvolutionPython+Notebook\utilities\fit3M2Parameters.py:90: RuntimeWarning: invalid value encountered in sqrt
  kk2p = 1/2 * ( -L1+S2/S1 -  np.sqrt((S1*L1-S2) **2-4*L3*S1)/S1 )
C:\Users\ovidiu\Dropbox\BurstDECONV_supp\code_BurstDECONV\DeconvolutionPython+Notebook\utilities\fit3M2Parameters.py:91: RuntimeWarning: invalid value encountered in sqrt
  kk1m = 1/2 * (S1-S2/S1 - (-S1 **2*L1+S1*S2+S1*L2-L3+S2 **2/S1-S3)/ np.sqrt((S1*L1-S2) **2-4*L3*S1))
C:\Users\ovidiu\Dropbox\BurstDECONV_supp\code_BurstDECONV\DeconvolutionPython+Notebook\utilities\fit3M2Parameters.py:92: RuntimeWarning: invalid value encountered in sqrt
  kk2m = 1/2 * (S1-S2/S1 + (-S1 **2*L1+S1*S2+S1*L2-L3+S2 **2/S1-S3)/ np.sqrt((S1